In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


# RAG 기본 파이프라인(1~8단계)

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [4]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader(file_path="./data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
print(f"문서의 페이지 수: {len(docs)}")

문서의 페이지 수: 23


In [5]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의 수: {len(split_documents)}")

분할된 청크의 수: 43


In [6]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [8]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성
vectorstore = FAISS.from_documents(split_documents, embedding=embeddings)

In [9]:
# 단계 5: 검색기(Retriever)생성
# 문서에 포함되어 있는 정보를 검색하고 생성
retriever = vectorstore.as_retriever()

In [10]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인
retriever.invoke("삼성전자가 자체 개발한 AI의 이름은?")

[Document(id='ce4b38c0-a288-4a86-8e6e-2b0e0037fbf4', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 12}, page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이

In [11]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in Korean.

    #Question:
    {question}
    #Context:
    {context}

    #Answer:"""
)

In [12]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM을 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [13]:
# 단계 8: 체인(Chain)생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력
question = "삼성전자가 자체 개발한 AI의 이름은?"
response = chain.invoke(question)
print(response)

삼성전자가 자체 개발한 AI의 이름은 '삼성 가우스'입니다.
